In [1]:
%load_ext autoreload
%autoreload 2


In [8]:
from sou.reasoning.run_reasoning import run_reasoning_loop, initialize_agents
from sou.reasoning.sequence import Sequence
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv(dotenv_path="../.env")



True

## Training data inference

In [9]:
from sou.reasoning.config import ReasoningSettings

sequence = Sequence(
    question="What is the capital of France?",
)

settings = ReasoningSettings(
    model_name="gpt-4",
    use_rag_agent=True,
    use_code_agent=True,
    forcing_search=True,
)

agents = initialize_agents(settings)


INFO:nano-graphrag:Creating working directory ./nano_graphrag_cache_2025-03-21-18:26:46
INFO:nano-graphrag:Load KV full_docs with 0 data
INFO:nano-graphrag:Load KV text_chunks with 0 data
INFO:nano-graphrag:Load KV llm_response_cache with 0 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './nano_graphrag_cache_2025-03-21-18:26:46/vdb_entities.json'} 0 data
#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 134B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.11-slim
#2 ...

#3 [auth] library/python:pull token for registry-1.docker.io
#3 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.11-slim
#2 DONE 1.4s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [1/2] FROM docker.io/library/python:3.11-slim@sha256:7029b0048

In [10]:
agents.keys()

dict_keys(['search_agent', 'code_agent', 'rag_agent'])

In [11]:
agents['search_agent'].run("test")

[SearchAgent] Gathering information for query: test


INFO:nano-graphrag:[New Docs] inserting 5 docs
INFO:nano-graphrag:[New Chunks] inserting 5 chunks
INFO:nano-graphrag:[Entity Extraction]...
INFO:nano-graphrag:Writing graph with 0 nodes, 0 edges


[SearchAgent] Analyzing data:
["Log In test 1 of 5 noun (1) ˈtest Synonyms of test 1 a : a means of testing: such as (1) : something (such as a series of questions or exercises) for measuring the skill, knowledge, intelligence, capacities, or aptitudes of an individual or group (2) : a procedure, reaction, or reagent used to identify or characterize a substance or constituent b : a positive result in such a test 2 a(1) : a critical examination, observation, or evaluation : trial specifically : the procedure of submitting a statement to such conditions or operations as will lead to its proof or disproof or to its acceptance or rejection a test of a statistical hypothesis (2) : a basis for evaluation : criterion b : an ordeal or oath required as proof of conformity with a set of beliefs c chiefly British : cupel 3 : a result or value determined by testing 4 : test match test 2 of 5 verb tested; testing; tests transitive verb 1 : to put to test or proof : try —often used with out 2 : to r

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
final_sequence = run_reasoning_loop(agents, settings, sequence)


In [ ]:
from sou.reasoning.run_reasoning import run_reasoning


accuracy_train = 0
final_results_train_complete = []
repetition = 5
for i in range(repetition):
    # Reset the finished flag for each sequence
    for j in range(len(active_sequences_train)):
        active_sequences_train[j]["finished"] = False
    # Run reasoning
    sequences_train = run_reasoning(
        filtered_data_train,
        active_sequences_train,
        forcing_research=True,
        code_mode_name="gpt-4o",
        general_model_args=model_args,
        tokenizer=tokenizer,
        tavily_client=tavily_client,
    )
    final_results_train = [
        active_sequences_train[i]["final_result"]
        for i in range(len(active_sequences_train))
    ]
    print(f"Final results: {final_results_train}")

    accuracy_train += sum(
        [
            1
            for i in range(len(final_results_train))
            if final_results_train[i] == train_df["answer"][i]
        ]
    ) / len(final_results_train)
    final_results_train_complete.append(final_results_train)

accuracy_train /= repetition
print(f"Accuracy on training set: {accuracy_train}")


## Test data inference

In [ ]:
from sou.reasoning.run_reasoning import run_reasoning


final_results_test_complete = []
repetition = 5
for i in range(repetition):
    for j in range(len(active_sequences_test)):
        active_sequences_test[j]["finished"] = False
    sequences_test = run_reasoning(
        filtered_data_test,
        active_sequences_test,
        forcing_research=True,
        code_mode_name="gpt-4o",
        general_model_args=model_args,
        tokenizer=tokenizer,
        tavily_client=tavily_client,
    )
    final_results_test = [
        active_sequences_test[i]["final_result"]
        for i in range(len(active_sequences_test))
    ]
    print(f"Final results: {final_results_test}")
    final_results_test_complete.append(final_results_test)


print("final_results_test_complete", final_results_test_complete)

In [ ]:
import numpy as np
import csv

final_results_test_complete = np.array(final_results_test_complete).T.tolist()


def save_predictions_to_csv(predictions_list, filename):
    """Save formatted predictions to CSV."""
    header = ["question_id"] + [f"prediction_{i + 1}" for i in range(5)]

    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for i, predictions in enumerate(predictions_list, start=1):
            writer.writerow([i] + predictions)


# Save the formatted predictions
save_predictions_to_csv(
    final_results_test_complete, filename="./output/predictions_test.csv"
)


In [ ]:
prompt = "What is the capital of France?"
final_sequence = run_reasoning_loop(prompt, reasoning_settings=settings)